# Download GEE Imagery

## Step-by-step tutorial

### Import libraries

In [1]:
import ee
import math
import geemap
import sys
import pandas as pd
from multiprocessing import Process, cpu_count 
from geetools import cloud_mask

In [2]:
Map = geemap.Map()
geometry =ee.Geometry.Polygon(
        [[[44.16644286751663, 42.600127655106185],
          [44.23510741829788, 42.5394457299407],
          [44.38067626595413, 42.48073042443687],
          [44.54547118782913, 42.456418303462],
          [44.72674560189163, 42.46452339297412],
          [44.87231444954788, 42.533374290609736],
          [44.90252685189163, 42.61023556954571],
          [44.80364989876663, 42.76367391712233],
          [44.56469726204788, 42.80398943064146],
          [44.16369628548538, 42.70315142463515]]]);
kazbegi = ee.FeatureCollection("users/andihanzl/kazbegi");
Map.centerObject(geometry, 10)
Map

Map(center=[42.62505591765608, 44.5423595888689], controls=(WidgetControl(options=['position'], widget=HBox(ch…

In [3]:
snow = ee.ImageCollection('MODIS/006/MOD10A1') \
            .merge(ee.ImageCollection('MODIS/006/MYD10A1')) \
            .select(['NDSI_Snow_Cover'],['Snow']) \
            .filterBounds(kazbegi) \
            .filter(ee.Filter.date('2000-01-01', '2002-12-31'))

res = snow.first().projection()

snow = snow.map(lambda image: image.reproject(res).clip(kazbegi))

# DEM
dataset = ee.Image('JAXA/ALOS/AW3D30/V2_2')
elevation = dataset.select(['AVE_DSM'], ['elevation'])
elekaz = elevation.clip(geometry).rename('DEM')
elekaz = ee.Image(elekaz)

In [4]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 22 17:20:38 2020

@author: kgn
"""
import sys
sys.path.append("C:/Users/andre/OneDrive/Dokumente/Masterarbeit/GEE/Download_Leonie/")
print(sys.path)
import ee
import numpy as np
from osgeo import gdal
from osgeo import osr

# fill all NA-values with a fill values of -999999 
def na_to_num(image):
    image_wo_na = image.unmask(-999999)
    return image_wo_na


# convert the gee objects into numpys and then into Geotiffs
def gee_to_numpy(img, bounds, poly, indexname):
    path_orig = 'original_files_'+indexname+'/'
    # get the lat lon and add the ndvi
    latlon = ee.Image.pixelLonLat().addBands(img)

    # apply reducer to list
    latlon = latlon.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=bounds,
      maxPixels=1e8,
      scale=500);
    
    # get data into three different arrays 
    data = np.array((ee.Array(latlon.get(indexname)).getInfo()))
    lats = np.array((ee.Array(latlon.get("latitude")).getInfo()))
    lons = np.array((ee.Array(latlon.get("longitude")).getInfo()))
    
   
    # get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)
     
    # get number of columns and rows from coordinates
    ncols = len(uniqueLons)    
    nrows = len(uniqueLats)
     
    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0] 
    xs = uniqueLons[1] - uniqueLons[0]
     
    # create an array with dimensions of image
    arr= np.zeros([nrows, ncols], np.float32) 
  
   # fill the array with values
    counter =0
    for r in range(0,len(arr),1):
        for p in range(0,len(arr[0]),1):
            if lats[counter] == uniqueLats[r] and lons[counter] == uniqueLons[p] and counter < len(lats)-1:
                counter+=1
                arr[len(uniqueLats)-1-r,p] = data[counter] # we start from lower left corner
                    
    # set -999999 to Na
    arr[arr == -999999] = 'nan' 
    
 #   if sat == 'l' or sat == 'm': # create Geotiffs from the landsat and MODIS images at the date 0 
        #SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    transform = (np.min(uniqueLons),xs,0,np.max(uniqueLats),0,-ys)
     
    # set the coordinate system
    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)
     
    # set driver
    driver = gdal.GetDriverByName('GTiff')
    
    outputDataset = driver.Create(path_orig+poly+".tif", ncols,nrows, 1,gdal.GDT_Float32)
     
    # add some metadata
     
    outputDataset.SetGeoTransform(transform)
    outputDataset.SetProjection(target.ExportToWkt())
    outputDataset.GetRasterBand(1).WriteArray(arr)
    outputDataset.GetRasterBand(1).SetNoDataValue(-999999)
    outputDataset = None
    return(outputDataset, arr)
   # else:
   #     return(arr)

['C:\\Users\\andre\\OneDrive\\Dokumente\\Masterarbeit\\GEE\\Download_Leonie', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\python37.zip', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\DLLs', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib', 'C:\\Users\\andre\\anaconda3\\envs\\starfm', '', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib\\site-packages', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib\\site-packages\\win32', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib\\site-packages\\win32\\lib', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib\\site-packages\\Pythonwin', 'C:\\Users\\andre\\anaconda3\\envs\\starfm\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\andre\\.ipython', 'C:/Users/andre/OneDrive/Dokumente/Masterarbeit/GEE/Download_Leonie/']


In [5]:
# -*- coding: utf-8 -*-
"""
Created on Thu Sep 10 09:19:52 2020

@author: kgn
"""

import ee
from geetools import cloud_mask
import pandas as pd
#import math


# Initialize the Earth Engine library.
ee.Initialize()

def download_imagery(collection, polygon, polygonname, number_of_processes):
    bounds = polygon
    #bounds = ee.FeatureCollection("users/leonieanina93/HessenBuff_Sued")
    
    # set the NAs to the numeric fillvalue of -999999
    #modis_wo_na = (modis_final.map(na_to_num))
    img_wo_na = na_to_num(collection)
    img_l = ee.Image(img_wo_na).clip(bounds)
    poly = str(polygonname)  #create a polygon label
    col_tif, arr_l = gee_to_numpy(img_l, bounds, poly, 'DEM')

In [6]:
download_imagery(elekaz, geometry, 'kazbegi_large_30m', 1)